In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 5

In [4]:
name = list[k].getName()
name

'RP28'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.8f,%.8f] " % ( resultsMC[4], resultsMC[5])


b = "[%.8f,%.8f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.8f,%.8f] " % ( resultSubset[4], resultSubset[5])


In [8]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.00000014,0.00000021] ', None],
 [None, None, None, '[0.00000014,0.00000021] ', None],
 [None, None, None, '[0.00000014,0.00000021] ', None],
 [None, None, None, '[0.00000014,0.00000021] ', None],
 [None, None, None, '[0.00000014,0.00000021] ', None],
 [None, None, None, '[0.00000014,0.00000021] ', None]]

In [9]:
l = np.array([[1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1], [1, 1, 1, a, 1, 1],[1, 1, 1, a, 1, 1]])

In [10]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité exacte de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  result[i][2]

results[0][3] = "-----------------"
results[1][3] = "-----------------"
results[2][3] = a

results[3][3] = c
 
results[4][3] = b


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2]->[ x1 * x2 - 146.14 ]


,Probabilité exacte de RP28,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,1.4917089e-07,1.7174278e-07,2.2571893e-08,-----------------,873399160,0.8201161049312867
FORM,1.4917089e-07,8.846209e-03,8.84606e-03,-----------------,101,-4.773065834020024
SORM,1.4917089e-07,1.0857472e-02,1.0857323e-02,"[0.00000014,0.00000021]",101,-4.862038684116551
FORM-IS,1.4917089e-07,1.027773e-07,4.6393588e-08,"[0.00000007,0.00000011]",133892,0.5072261275535662
SUBSET,1.4917089e-07,8.87e-08,6.047089e-08,"[0.00000008,0.00000012]",80000,0.3921377292411602


In [11]:
df.to_csv('problem28.csv')